In [ ]:
import os
import cv2
import pandas as pd

In [ ]:
def dibujar_puntos(imagen, punto_referencia, punto_marcado):
    cv2.circle(imagen, punto_referencia, 5, (0, 0, 255), -1)  # Punto de referencia en rojo
    cv2.circle(imagen, punto_marcado, 5, (255, 0, 0), -1)  # Punto marcado en azul

In [ ]:
def crear_matriz(num_columnas, num_filas):
    return [[0 for _ in range(num_columnas)] for _ in range(num_filas)]

In [ ]:
def calcular_centro_grilla(x_ref, y_ref, pixeles_por_mm, num_filas):
    centro_x = x_ref
    centro_y = y_ref  # Ajustar para que la grilla se dibuje desde el punto de referencia hacia arriba
    tamanio_celda = pixeles_por_mm
    return centro_x, centro_y, tamanio_celda

In [ ]:
def dibujar_grilla(imagen, centro_x, centro_y, tamanio_celda, num_columnas, num_filas, matriz_grilla):
    for i in range(num_filas + 1):
        y = int(centro_y - i * tamanio_celda)
        cv2.line(imagen, (int(centro_x - (num_columnas // 2) * tamanio_celda), y), (int(centro_x + (num_columnas // 2) * tamanio_celda), y), (0, 255, 0), 1)
    for j in range(num_columnas + 1):
        x = int(centro_x - (num_columnas // 2) * tamanio_celda + j * tamanio_celda)
        cv2.line(imagen, (x, int(centro_y - num_filas * tamanio_celda)), (x, int(centro_y)), (0, 255, 0), 1)

    for y in range(num_filas):
        for x in range(num_columnas):
            valor = matriz_grilla[y][x]
            coord_x = int(centro_x - (num_columnas // 2) * tamanio_celda + x * tamanio_celda + tamanio_celda // 2)
            coord_y = int(centro_y - y * tamanio_celda - tamanio_celda // 2)
            cv2.putText(imagen, str(valor), (coord_x - 10, coord_y + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

In [ ]:
def obtener_posicion_grilla(x_marcado, y_marcado, centro_x, centro_y, tamanio_celda, num_columnas, num_filas):
    grilla_x = int((x_marcado - centro_x + (num_columnas // 2) * tamanio_celda) / tamanio_celda)
    grilla_y = int((centro_y - y_marcado) / tamanio_celda)
    return grilla_x, grilla_y

In [ ]:
def procesar_imagen_individual(ruta_imagen, punto_referencia, puntos_marcados, pixeles_por_3mm, directorio_salida):
    
    imagen = cv2.imread(ruta_imagen)
    if imagen is None:
        print(f"No se pudo cargar la imagen: {ruta_imagen}")
        return None

    pixeles_por_mm = pixeles_por_3mm
    num_columnas, num_filas = 8, 6

    dibujar_puntos(imagen, punto_referencia, punto_referencia)  # Dibujar el punto de referencia

    centro_x, centro_y, tamanio_celda = calcular_centro_grilla(punto_referencia[0], punto_referencia[1], pixeles_por_mm, num_filas)

    matriz_grilla = crear_matriz(num_columnas, num_filas)  # Inicializar la matriz sin puntos marcados

    for punto_marcado in puntos_marcados:
        dibujar_puntos(imagen, punto_referencia, punto_marcado)
        grilla_x, grilla_y = obtener_posicion_grilla(punto_marcado[0], punto_marcado[1], centro_x, centro_y, tamanio_celda, num_columnas, num_filas)

        # Verificar que los índices están dentro de los límites
        if 0 <= grilla_x < num_columnas and 0 <= grilla_y < num_filas:
            matriz_grilla[grilla_y][grilla_x] = 1  # Marcar el punto en la matriz
        else:
            print(f"Punto fuera de la grilla: {punto_marcado} -> ({grilla_x}, {grilla_y})")

    dibujar_grilla(imagen, centro_x, centro_y, tamanio_celda, num_columnas, num_filas, matriz_grilla)

    # Asegurarse de que directorio_salida es un directorio
    if not os.path.exists(directorio_salida):
        os.makedirs(directorio_salida)

    ruta_salida = os.path.join(directorio_salida, os.path.basename(ruta_imagen))
    cv2.imwrite(ruta_salida, imagen)
    
    return matriz_grilla

In [ ]:
def procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida, ruta_salida_matriz):
    conjunto_datos = pd.read_csv(ruta_conjunto_datos)
    os.makedirs(directorio_salida, exist_ok=True)

    matrices = []

    for indice, fila in conjunto_datos.iterrows():
        nombre_imagen = fila["imagen"]
        ruta_imagen = os.path.join('..', 'imagenes', 'seleccionadas', 'final', nombre_imagen)

        punto_referencia = (int(fila['punto_referencia_x']), int(fila['punto_referencia_y']))
        mm_por_pixel = fila['mm_por_pixel']

        puntos_marcados = []
        for i in range(1, 16):
            coord_x = f'coordenada_x_punto {i}'
            coord_y = f'coordenada_y_punto {i}'

            if coord_x in fila and coord_y in fila:
                punto_marcado = (int(fila[coord_x]), int(fila[coord_y]))
                puntos_marcados.append(punto_marcado)
            else:
                print(f"No se encontraron coordenadas para el punto {i}")

        if puntos_marcados:
            matriz_grilla = procesar_imagen_individual(ruta_imagen, punto_referencia, puntos_marcados, mm_por_pixel, directorio_salida)
            if matriz_grilla is not None:
                matrices.append({
                    'imagen': nombre_imagen,
                    'matriz': matriz_grilla
                })

    # Guardar las matrices en un archivo CSV
    df_matrices = pd.DataFrame(matrices)
    df_matrices.to_csv(ruta_salida_matriz, index=False)

    print("\nProcesamiento completado.")
    print(f"Directorio de salida: {os.path.abspath(directorio_salida)}")
    print(f"Matriz guardada en: {os.path.abspath(ruta_salida_matriz)}")


In [ ]:

ruta_conjunto_datos = '../datos/df.csv'
directorio_salida = '../imagenes/procesadas/'
ruta_salida_matriz = '../datos/matriz_grilla_final.csv'
procesar_conjunto_datos(ruta_conjunto_datos, directorio_salida, ruta_salida_matriz)